In [ ]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import yaml
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch import multiprocessing

from mars_dataset import MARSDataset

%matplotlib inline

In [ ]:
torch.cuda.empty_cache()
torch.set_default_tensor_type('torch.cuda.FloatTensor')
multiprocessing.set_start_method('spawn')

In [ ]:
with open('detector_config.yaml', 'r') as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.Loader)

In [ ]:
data = pd.read_json('../MARS-data-tagger/recordings.json')
data.dropna(inplace=True)
data['y'] = data['label'].apply(lambda x: 1 if x != 'no_whale' else 0)

from tutorial: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [ ]:
X_train, X_val = train_test_split(data, 
                                  test_size=config['TEST_SIZE'], 
                                  random_state=config['RANDOM_STATE'],
                                  stratify=data['y'],
                                  )
X_train['y'].value_counts(), X_val['y'].value_counts()

trainset = MARSDataset(X_train)
trainloader = DataLoader(trainset,
                         batch_size=config['N_BATCH'], 
                         shuffle=True, 
                         num_workers=2,
                         generator=torch.Generator(device='cuda'))

testset = MARSDataset(X_val)
testloader = DataLoader(testset, 
                        batch_size=config['N_BATCH'], 
                        shuffle=True, 
                        num_workers=2,
                        generator=torch.Generator(device='cuda'))

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5,5))
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc1 = nn.Linear(206976, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

net = Net()
net.cuda()
print(net)

In [ ]:
for epoch in range(config['N_EPOCHS']):
    print(f'Epoch: {epoch+1} / {config["N_EPOCHS"]}')
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(tqdm(trainloader), 0):
        
        # move data to GPU
        inputs = inputs.unsqueeze(1).to(torch.device('cuda'))        
        labels = labels.to(torch.device('cuda'))
        
        # check that inputs are valid
        assert ~torch.isnan(inputs).any(), f'input error (nan): {inputs}'
        assert ~torch.isinf(inputs).any(), f'input error (inf): {inputs}'
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # generate statistics
        running_loss += loss.item()

    print(f'Loss: {running_loss / (i+1)}')

print('Finished Training')